# Propositional Logic Clause Parser (PLCParser)

This library is used to parse string input that contains five fundamental [propositional logic](https://en.wikipedia.org/wiki/Logical_conjunction) symbols: 

1. $AND$
2. $OR$
3. $NOT$
4. $XOR$
5. $XAND$

$IS$ operator can also be used, but one should rather not use, because it is mostly useless tautology. (IS A) is same as (A) anyway.

Library takes a string input and produces a multidimensional list of given literals with [parseInput](#Simple-example) function. Structure of the nested output list is created to contain all information required to use it programmatically for boolean operations. [Validate](#Validate) method is used to check if given input is in correct syntax form. You can [deformat](#Deformat) well formatted list structure back to literal representation. Furthermore [evaluation](#Evaluate) of the given clause or structure can be done with an optional interpretation table.

<blockquote>Library is implemented in three languages: [Python](#Python-version), [PHP](#PHP-version) and [Javascript](#Javascript-version). For both PHP and Javascript version, see [PLCParser demo application](https://plcparser.herokuapp.com/) deployed in Heroku for live testing.</blockquote>

## Python version

### Simple example

In [1]:
# load library
from pyPLCParser import parseInput
# set input
i = "(1 AND 2)"
# parse input
o = parseInput(i)
# print output
print(o)

(True, [['1', '2']])


From the output <code>(True, [['1', '2']])</code> we can read that mutual boolean starting point for the interpretation of the input is *and* which is *True*. This is the default starting point. You could also just simply input: <code>(1 2)</code> to get the same result.

To demonstrate opposite starting point, let us use | character for OR operator. You can use either $AND, OR, NOT, XOR, XAND$ keywords or single characters on the clause:  $\&$, $|$, $\\!$, ^, $+$ respectively. Also corresponding mathematical symbols are accepted: $∧$, $∨$, $¬$,  $⊕$, $⊖$.

In [2]:
i = "(1 | 2)"
parseInput(i)

(False, [['1', '2']])

But say, you have a more complex nested clause in your hands, what is the outcome?

In [3]:
i = "(A or (B C (D E)))"
parseInput(i)

(False, [['A', ['B', 'C', ['D', 'E']]]])

From the output we can read that the first level must be interpreted as an OR clause (<code>False</code>). It is actually the only mandatory boolean keyword you need to use here because the next level is automaticly regarded as an AND clause. Thus the deepest level (D E) is again an OR clause.

If you don't give a keyword for the first level <code>(A or (B C (D E)))</code>, it is assumed to be AND, and the next level is OR, and the next AND, and so forth. 

This represents the important mutual boolean change of the interpretation of the nested levels. Most of the meaningful boolean clauses can be represented in this way. For example, if you consider this:

<code>((A or B) and (C and D))</code>

it would be inappropriate in syntax, because <code>(C and D)</code> is actually interpreted <code>(C or D)</code> by the parser. But right way would be to write it:

<code>((A or B) and C and D)</code>

or just

<code>((A B) C D)</code>

which of course looks a bit ambiguous at first, but is clear for the parser. PLCParser <u>does not</u> try to deformalize and interpret the [boolean operator precedence]( http://stackoverflow.com/questions/12494568/boolean-operators-precedence) in any other way. You have to decide and choose correctly format nested set / parentheses to get the right results. [Order of the precedence](https://en.wikipedia.org/wiki/Logical_connective#Order_of_precedence) differs by authors anyway.

### Literals

In above examples only single word letters were used. That is ok as long as they are not reserved keywords or the chosen parenthesis and literal wrapper characters. To use sentences that contain spaces and special characters it is safer to make it this way:

In [4]:
i = "('Queen Elizabeth' & 'Philip, Duke of Edinburgh')"
parseInput(i)

(True, [['Queen Elizabeth', 'Philip, Duke of Edinburgh']])

By default literals are expected to be wrapped with single ' or double " quotes. Parentheses are assumed to be ().

If default parentheses and literal wrappers are not suitable, you can change them and parse input accordingly:

In [5]:
from pyPLCParser import PLCParser

c = PLCParser(parentheses=['[', ']'], wrappers=['´'])

i = "[´Use´ ´as you´ wish]"

c.parse(i)

(True, [['Use', 'as you', 'wish']])

### Negation, XOR and XAND

Using negation (NOT / !), exclusive or (XOR / ^) or exclusive and (XAND / +) keywords do shape the structure of the output. NOT will add -1 value to the result before the item or node. Similarly XOR will add -2 and XAND will add -3 value to the list.

Negation can be done for item or to the group of items:

In [6]:
parseInput("""

(!A and !(B or C))

""")

(True, [[-1, 'A', -1, ['B', 'C']]])

Again same inut could be written many ways:

<code>(!A and !B and !C) -> (!A !B !C)</code> 

<code>(!(A or B or C)) -> (!(A B C))</code> 

Note that the meaning of <code>!(A B C)</code> however is different. It means the negation of a group where all items A, B and C exists. If only one or two of the group items existed, then negation wouldn't be true.

This brings us to the XOR operator. XOR is an exclusive OR, which states that either A or B should exist, but not both at the same time. Same behaviour could be achieved by OR, AND, and NOT clause groups. Let us demonstrate it by few examples:

In [7]:
# xor logic -> one of the group, but not all
i1 = '(^(A or B))' # or just (^(A B))
o1 = parseInput(i1)
print(o1)

# xor logic with and, or, and not operators #1
i2 = '((A or B) and (!(A and B)))'
o2 = parseInput(i2)
print(o2)

# xor logic with and, or, and not operators #2
i3 = '((A and !B) or (!A and B))'
o3 = parseInput(i3)
print(o3)

# xor logic with and, or, and not operators #3
i4 = '((A or !B) and (!A or B))'
o4 = parseInput(i4)
print(o4)

(True, [[-2, ['A', 'B']]])
(True, [[['A', 'B'], [-1, ['A', 'B']]]])
(False, [[['A', -1, 'B'], [-1, 'A', 'B']]])
(True, [[['A', -1, 'B'], [-1, 'A', 'B']]])


Because XOR is a group operator in PLCParser, you can use it only before nested groups. That also means there can be more than two items in a group:

In [8]:
# xor logic -> one of the group, but not two or all
i1 = '(^(A or B or C))'
o1 = parseInput(i1)
print(o1)

# xor logic with and, or, and not operators #1
i2 = '((A or B or C) and (!(A and B and C)))'
o2 = parseInput(i2)
print(o2)

# xor logic with and, or, and not operators #2
i2 = '((A or !B or !C) and (!A or !B or C) and (!A or B or !C))'
o2 = parseInput(i2)
print(o2)

(True, [[-2, ['A', 'B', 'C']]])
(True, [[['A', 'B', 'C'], [-1, ['A', 'B', 'C']]]])
(True, [[['A', -1, 'B', -1, 'C'], [-1, 'A', -1, 'B', 'C'], [-1, 'A', 'B', -1, 'C']]])


Apparently using XOR can save a lot of space!

Exclusive and (XAND) works exacly same way, except that the group after can have one or more True values, but not all True. This differs from XOR that on the latter group can have only one True value. To detect any other number of True occurences you need to combine clause with any of the five given propositions.

In [9]:
# xor logic -> one of the group, but not two or all
i1 = '(+(A or B or C))'
o1 = parseInput(i1)
print(o1)

(True, [[-3, ['A', 'B', 'C']]])


### Validate

ValidateInput method is used to validate given clause in string format. It can be used to roughly check that parentheses and literals are correctly formed. Then it is safer to use parseInput and evaluateInput functions.

In [10]:
from pyPLCParser import validateInput

print (validateInput('(A B))'), validateInput('(A B)'))

False True


### Deformat

Of cource it is good to have a method to deformat native list structure back to the string clause representation.

With an optional argument, one can user special character abbreviations for logic operators. By default formal keywords are used on output. Lastly one can decide to use only the first level keyword if output should be as concise as possible.

In [11]:
from pyPLCParser import PLCParser
c = PLCParser()
i = [True, [['A', 'B']]]
c.deformat(i, short=False, first=False, latex=False)

"( ( 'A' and 'B' ) )"

### Evaluate

EvaluateInput function takes propositional logic clause in string or array format and evaluates it. Usually, if everything is correct on input, output is either true or false.

In [12]:
from pyPLCParser import evaluateInput

i = "(1 or 0)"
o = evaluateInput(i)
print("%s => %s" % (i, o))

i = "(1 and 0)"
o = evaluateInput(i)
print("%s => %s" % (i, o))

i = "(true and false)"
o = evaluateInput(i)
print("%s => %s" % (i, o))

i = "(A and B)"
o = evaluateInput(i, {'A': True, 'B': False})
print("%s => %s" % (i, o))

(1 or 0) => True
(1 and 0) => False
(true and false) => False
(A and B) => False


## PHP version

PHP version of the PLCParser class is practically same as Python having same API methods and functionality. For example parseInput is called something like this:

In [13]:
%%html

require_once('./src/elonmedia/plcparser/php/PLCParser.php')
print_r(PLCParser::parseInput("('A' or 'B')"));

In [14]:
%%html
Array
(
    [0] => 
    [1] => Array
        (
            [0] => Array
                (
                    [0] => Array
                        (
                            [0] => A
                            [2] => B
                        )

                )

        )

)

## Javascript version

Same applies to Javascript library. You need to include library first and then you can use same API methods (validateInput, parseInput, EvaluateInput, deformatInput) or build object fromthe  PLCParser "class" for more specific usage:

In [15]:
%%html

<script src="./dist/PLCParser.min.js"></script>

<script>

var v = PLCParser.validateInput("(A and B (C | D) !F)")

console.log(v)

var p = PLCParser.parseInput("(A and B (C | D) !F)")

console.log(p)

var e = PLCParser.evaluateInput("(true or false)")

console.log(e)

var d = PLCParser.deformatInput([true, [-1, ['A', 'B']]])

console.log(d)

</script>

For both PHP and Javascript version, see [PLCParser demo application](https://plcparser.herokuapp.com/) deployed in Heroku for live testing.

## The [MIT](https://choosealicense.com/licenses/mit/) License
Copyright &copy; 2017 Marko Manninen